In [ ]:
!pip install scrapy

In [2]:
import numpy as np
import pandas as pd
import time 
import requests
from scrapy.http import TextResponse 

In [3]:
url="http://books.toscrape.com/"
base_url = "http://books.toscrape.com/catalogue/"
base_url1 = "http://books.toscrape.com/"
page = requests.get(url)
response = TextResponse(url=page.url,body=page.text,encoding="utf-8")

In [81]:
# Defining functions for title, rating, price, book page, book image,
# availability in order to scrape the first page of the website.
# In addition, creating functions for individual page, genre and descriptions
# for further use. 

def get_title(url):
  page = requests.get(url)
  response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
  title = response.css("a[title]::attr(title)").extract()
  return title
  
def get_rating(url):
  page = requests.get(url)
  response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
  rating =response.css("p[class^='star-rating']::attr(class)").extract()
  return rating 

def get_price(url):
  page = requests.get(url)
  response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
  price = response.css("p.price_color::text").extract()
  price = [i.replace("Â£", " ") for i in price]
  return price

def get_book_page(url):
  page = requests.get(url)
  response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
  book_page = [base_url +i for i in response.css("a[title]::attr(href)").extract()]
  return book_page 

def get_book_picture(url):
  page = requests.get(url)
  response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
  book_picture_url = response.css("img::attr(src)").extract()
  book_picture = [base_url1 +i for i in book_picture_url]
  return book_picture

def get_availabilities(url):
  page = requests.get(url)
  response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
  availability = response.xpath("//p[contains(@class,'stock')]/text()[2]").re("\w+.+\w")
  return availability

def individual_page_url(url):
    page = requests.get(url)
    response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
    individual_page = [base_url+'catalogue/'+i for i in response.css('article.product_pod > div.image_container > a::attr(href)').extract()] #or title 
    return individual_page

def book_genre(url): 
  page = requests.get(url)
  response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
  genre = response.xpath("//ul[@class = 'breadcrumb']/li[3]/a/text()").extract_first()
  return genre

def book_description(url):
    page = requests.get(url)
    response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
    description = response.css("article.product_page > p::text").extract_first()
    return description


In [43]:
# Writing a code all_pages in order to scrape all of the pages.
# The amount of pages is equal to 50.
all_pages = [base_url+"catalogue/page-{}.html".format(i) for i in range(1,51)]

In [44]:
# Creating empty lists and extending them with first scraped page in order 
# to receive all the necessary data for all pages. 
all_titles = []
all_ratings = []
all_prices = []
all_book_pages = []
all_book_pictures = []
all_availabilities = []
individual_page = []
for i in all_pages:
    all_titles.extend(get_title(i))
    all_ratings.extend(get_rating(i))
    all_prices.extend(get_price(i))
    all_book_pages.extend(get_book_page(i))
    all_book_pictures.extend(get_book_picture(i))
    all_availabilities.extend(get_availabilities(i))
    individual_page.extend(individual_page_url(i))
    time.sleep(1)

In [92]:
# Creating separately empty lists for genre and descriptions and 
# append them to our previously made functions and scraping usinf individual_page. 
books_genre = []
books_descriptions = []
for i in individual_page:
    books_genre.append(book_genre(i))
    books_descriptions.append(book_description(i))
    time.sleep(1)

In [121]:
# Creating a dataframe and turning it into csv file.
df = pd.DataFrame(np.column_stack([all_titles, all_ratings, all_prices, all_book_pages, all_book_pictures, all_availabilities, individual_page, books_genre, books_descriptions]), columns = ['Titles', 'Ratings', 'Prices', 'Book_Pages', 'Book_Pictures', 'Availabilities', 'Individual_Page', 'Books_Genre', 'Books_Descriptions'])

In [123]:
 df.to_csv('all_books.csv', index=False) 

In [124]:
data = pd.read_csv("all_books.csv")

In [139]:
# Question 1. Calculating the average price of all books 
# using mean() function.  
data["Prices"].mean()

35.07034999999999

In [147]:
# Question 2. Grouping prices by genre, calculating the mean, and using
# sort_values() function in order to see on average which genre is the most expensive one. 
# Therefore, the answer is "Suspense."
data['Prices'].groupby(data['Books_Genre']).mean().sort_values(ascending = False).head(1)

Books_Genre
Suspense    58.33
Name: Prices, dtype: float64

In [ ]:
# Question 3. In order to calculate the correlation, we need
# to replace our string with numbers. Afterwards, we can calculate 
# the correlation and see whether more expensive books are also rated 
# higher. The answer is 0.028, which means that there is a very weak positive relationship
# between our variables. However, it's not enough to say that those books with higher prices 
# have higher ratings. 
data.Ratings[data.Ratings == 'star-rating One'] = 1
 data.Ratings[data.Ratings == 'star-rating Two'] = 2
data.Ratings[data.Ratings == 'star-rating Three'] = 3
data.Ratings[data.Ratings == 'star-rating Four'] = 4
data.Ratings[data.Ratings == 'star-rating Five'] = 5

In [137]:
data["Prices"].corr(data["Ratings"])

0.028166239485873015